# Simple Action Selection vs Upper Confidence Bound Action Selection

In [ ]:
using Distributions
using Gadfly
using DataFrames


Simple Action Selection:

With probability $1 - \varepsilon$:

$$A_t ={arg\,max}_{a} \left[Q_t(a)\right]$$

Upper Confidence Bound Action Selection:

$$A_t ={arg\,max}_{a} \left[Q_t(a) + c\sqrt{\frac{\log{t}}{N_t(a)}} \right]$$

In [ ]:
seed = 20

In [ ]:
n = 1000
plot(
    layer(x=1:n, y=8 * sqrt(log(1:n) / 1000), Geom.line, Theme(default_color=colorant"red")),
    layer(x=1:n, y=16 * sqrt(log(1:n) / 1000), Geom.line, Theme(default_color=colorant"green")),
    layer(x=1:n, y=32 * sqrt(log(1:n) / 1000), Geom.line, Theme(default_color=colorant"blue")),
)
  

In [ ]:
n = 1000
plot(
    layer(x=1:n, y=16 * sqrt(log(1:n) / 100), Geom.line, Theme(default_color=colorant"red")),
    layer(x=1:n, y=16 * sqrt(log(1:n) / 1000), Geom.line, Theme(default_color=colorant"green")),
    layer(x=1:n, y=16 * sqrt(log(1:n) / 10000), Geom.line, Theme(default_color=colorant"blue")),
)

In [ ]:
n = 1000
plot(
    layer(x=1:n, y=8 * sqrt(log(1:n) / 100), Geom.line, Theme(default_color=colorant"red")),
    layer(x=1:n, y=16 * sqrt(log(1:n) / 1000), Geom.line, Theme(default_color=colorant"green")),
    layer(x=1:n, y=32 * sqrt(log(1:n) / 10000), Geom.line, Theme(default_color=colorant"blue")),
)

In [ ]:
# Creates a random bandit.
function create_bandit(mean, std)
    f() = rand(Normal(mean, std))
end
# Returns an array containing the given number of random bandits
function init_bandits(bandit_number)
    buckets = []
    for i = 1:bandit_number
        push!(buckets, create_bandit(rand(10:100), rand(1:5)))
    end
    buckets
end
# Returns an array containing the indexes with the max value in the given array.
function arg_max(array)
    max = maximum(array)
    indexes = []
    for i = 1:length(array)
        if max == array[i]
            push!(indexes, i)
        end
    end
    return indexes
end
# Returns an array containing the potential of each action using the Upper Confidence Bound Action Selection formula.
function get_action_potential(Q, N, t, c=1)
    size = length(Q)
    potential = zeros(Float32, size)
    for i = 1:size
        if t > 0
            if N[i] > 0
                potential[i] = Q[i] + c * sqrt(log(t) / N[i])
            else
                potential[i] = Inf
            end
        end
    end
    return potential
end
# Returns a random element from the array.
function get_random(array)
   array[rand(1:length(array))] 
end
# Returns a simple selection function with the given epsilon cut.
function simple_selection_factory(epsilon)
    f(Q, N, i) = if (rand() > epsilon) get_random(arg_max(Q)) else rand(1:length(Q)) end
end
# Returns a ucb selection function with the given exploration factor.
function ucb_selection_factory(exploration)
    f(Q, N, i) = get_random(arg_max(get_action_potential(Q, N, i, exploration)))
end
# This function runs the multi-armed bandits algorithm with the given selection function.
function generic_bandits_algorithm(bandits, simulations, selection)
    bandits_number = length(bandits)
    N = zeros(Int, bandits_number)
    Q = zeros(Float32, bandits_number)
    reward_average = zeros(Float32, simulations)
    for i = 1:simulations
        A = selection(Q, N, i)
        R = bandits[A]()
        N[A] += 1
        Q[A] = Q[A] + (R - Q[A]) / N[A]
        if i > 1
            reward_average[i] = reward_average[i - 1] + (R - reward_average[i - 1]) / i
        else
            reward_average[i] = R
        end
    end
    return reward_average
end
# Decorator for the Simple Bandit Selection algoritm implementation.
function simple_bandit_algorithm(bandits, simulations, epsilon)
    return generic_bandits_algorithm(bandits, simulations, simple_selection_factory(epsilon))
end
# Decorator for the Upper Confidence Bound Action Selection algoritm implementation.
function ucba_selection_algorithm(bandits, simulations, exploration)
    return generic_bandits_algorithm(bandits, simulations, ucb_selection_factory(exploration))
end

In [ ]:
srand(seed)
simulations = 10 # try [10,100,1000,10000]
number_of_bandits = 10
bandits = init_bandits(number_of_bandits)

x = 1:simulations
df1 = DataFrame(x=x, y=simple_bandit_algorithm(bandits, simulations, 0.1), label="epsilon=0.1")
df2 = DataFrame(x=x, y=simple_bandit_algorithm(bandits, simulations, 0.01), label="epsilon=0.01")
df3 = DataFrame(x=x, y=simple_bandit_algorithm(bandits, simulations, 0), label="epsilon=0")
df4 = DataFrame(x=x, y=ucba_selection_algorithm(bandits, simulations, 3), label="c=3")

df = vcat(df1, df2, df3, df4)

p = plot(df, x="x", y="y", color="label", Geom.line,
         Scale.color_discrete_manual("blue","red", "black", "green"))



In [ ]:
srand(seed)
simulations = 10 # try [10,100,1000,10000]
range = []
for i = -6:5
    push!(range, 2.0^i)
end
simple = []
ucb = []
for i=1:length(range)
    if(range[i] < 1)
        push!(simple, simple_bandit_algorithm(bandits, simulations, range[i])[simulations])
    end
end
for i=1:length(range)
    if(range[i] > 1)
        push!(ucb, ucba_selection_algorithm(bandits, simulations, range[i])[simulations])
    end
end
simple_df = DataFrame(x=range[1:length(simple)], y=simple, label="simple bandit")
ucb_df = DataFrame(x=range[length(range)-length(ucb):length(range)-1], y=ucb, label="ucba selection")
df = vcat(simple_df, ucb_df)
p = plot(df, x="x", y="y", color="label", Geom.line,Scale.x_log2(minvalue=5))